In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json
from pandas import json_normalize


# Display
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)  # Adjust as needed
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)


## Data path
csv_path = "/Users/meenu/Documents/python_workspace/fairmoney/data/raw/credit.csv"

In [5]:
df_csv = pd.read_csv(csv_path, index_col=0)
print(df_csv.shape)
display(df_csv.head())
df_csv.info()

(1000, 22)


,checking_balance,months_loan_duration,credit_history,purpose,amount,savings_balance,employment_length,installment_rate,personal_status,other_debtors,residence_history,property,age,installment_plan,housing,existing_credits,default,dependents,telephone,foreign_worker,job,gender
0,-43.0,6,critical,radio/tv,1169,NaN,13 years,4,single,none,6 years,real estate,67,none,own,2,0,1,2.349340e+09,yes,skilled employee,male
1,75.0,48,repaid,radio/tv,5951,89.0,2 years,2,NaN,none,5 months,real estate,22,none,own,1,1,1,NaN,yes,skilled employee,female
2,NaN,12,critical,education,2096,24.0,5 years,2,single,none,4 years,real estate,49,none,own,1,0,2,NaN,yes,unskilled resident,male
3,-32.0,42,repaid,furniture,7882,9.0,5 years,2,single,guarantor,13 years,building society savings,45,none,for free,1,0,2,NaN,yes,skilled employee,male
4,-23.0,24,delayed,car (new),4870,43.0,3 years,3,single,none,13 years,unknown/none,53,none,for free,2,1,2,NaN,yes,skilled employee,male


<class 'pandas.core.frame.DataFrame'>
Index: 1000 entries, 0 to 999
Data columns (total 22 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   checking_balance      606 non-null    float64
 1   months_loan_duration  1000 non-null   int64  
 2   credit_history        1000 non-null   object 
 3   purpose               1000 non-null   object 
 4   amount                1000 non-null   int64  
 5   savings_balance       817 non-null    float64
 6   employment_length     938 non-null    object 
 7   installment_rate      1000 non-null   int64  
 8   personal_status       690 non-null    object 
 9   other_debtors         1000 non-null   object 
 10  residence_history     870 non-null    object 
 11  property              1000 non-null   object 
 12  age                   1000 non-null   int64  
 13  installment_plan      1000 non-null   object 
 14  housing               1000 non-null   object 
 15  existing_credits      1000 

In [6]:
## Check missing data
df_csv.isnull().sum().sort_values(ascending=False)

telephone               596
checking_balance        394
personal_status         310
savings_balance         183
residence_history       130
employment_length        62
installment_plan          0
job                       0
foreign_worker            0
dependents                0
default                   0
existing_credits          0
housing                   0
property                  0
age                       0
months_loan_duration      0
other_debtors             0
installment_rate          0
amount                    0
purpose                   0
credit_history            0
gender                    0
dtype: int64

In [7]:
df_csv['default'].value_counts()

default
0    700
1    300
Name: count, dtype: int64

### understand Each variable 

In [8]:
df_csv.head()

,checking_balance,months_loan_duration,credit_history,purpose,amount,savings_balance,employment_length,installment_rate,personal_status,other_debtors,residence_history,property,age,installment_plan,housing,existing_credits,default,dependents,telephone,foreign_worker,job,gender
0,-43.0,6,critical,radio/tv,1169,NaN,13 years,4,single,none,6 years,real estate,67,none,own,2,0,1,2.349340e+09,yes,skilled employee,male
1,75.0,48,repaid,radio/tv,5951,89.0,2 years,2,NaN,none,5 months,real estate,22,none,own,1,1,1,NaN,yes,skilled employee,female
2,NaN,12,critical,education,2096,24.0,5 years,2,single,none,4 years,real estate,49,none,own,1,0,2,NaN,yes,unskilled resident,male
3,-32.0,42,repaid,furniture,7882,9.0,5 years,2,single,guarantor,13 years,building society savings,45,none,for free,1,0,2,NaN,yes,skilled employee,male
4,-23.0,24,delayed,car (new),4870,43.0,3 years,3,single,none,13 years,unknown/none,53,none,for free,2,1,2,NaN,yes,skilled employee,male


In [11]:
# Suppose df is your DataFrame
object_cols = df_csv.select_dtypes(include=['object']).columns.tolist()
numeric_cols = df_csv.select_dtypes(exclude=['object']).columns.tolist()

print("Object columns:", object_cols)
print("Numeric columns:", numeric_cols)

Object columns: ['credit_history', 'purpose', 'employment_length', 'personal_status', 'other_debtors', 'residence_history', 'property', 'installment_plan', 'housing', 'foreign_worker', 'job', 'gender']
Numeric columns: ['checking_balance', 'months_loan_duration', 'amount', 'savings_balance', 'installment_rate', 'age', 'existing_credits', 'default', 'dependents', 'telephone']


In [36]:
df_csv['savings_balance'].value_counts()

savings_balance
89.0      14
30.0      12
13.0      12
57.0      11
80.0      11
          ..
876.0      1
790.0      1
265.0      1
6195.0     1
128.0      1
Name: count, Length: 302, dtype: int64

In [22]:
df_csv['credit_history'] = df_csv['credit_history'].replace({
    'fully repaid this bank': 'repaid',
    'fully repaid': 'repaid'
})


import re
# Convert all values to months
def convert_to_months(x):
    if pd.isnull(x):
        return None
    x = str(x).lower().strip()
    if 'month' in x:
        num = int(re.search(r'\d+', x).group())
        return num
    elif 'year' in x:
        num = int(re.search(r'\d+', x).group())
        return num * 12
    else:
        return None

df_csv['employment_months'] = df_csv['employment_length'].apply(convert_to_months)

# Define buckets
def bucket_employment(months):
    if months is None:
        return 'Unknown'
    if months < 12:
        return '<1 year'
    elif 12 <= months < 36:
        return '1-3 years'
    elif 36 <= months < 84:
        return '3-7 years'
    else:
        return '7+ years'
    
    
# Convert residence history to months
def convert_residence_to_months(x):
    if pd.isnull(x):
        return None
    x = str(x).lower().strip()
    if 'month' in x:
        num = int(re.search(r'\d+', x).group())
        return num
    elif 'year' in x:
        num = int(re.search(r'\d+', x).group())
        return num * 12
    else:
        return None

df_csv['residence_months'] = df_csv['residence_history'].apply(convert_residence_to_months)

# Define buckets
def bucket_residence(months):
    if months is None:
        return 'Unknown'
    if months < 6:
        return '<6 months'
    elif 6 <= months < 12:
        return '6-12 months'
    elif 12 <= months < 36:
        return '1-3 years'
    elif 36 <= months < 84:
        return '3-7 years'
    else:
        return '7+ years'

In [24]:
df_csv['employment_bucket'] = df_csv['employment_months'].apply(bucket_employment)
df_csv['residence_bucket'] = df_csv['residence_months'].apply(bucket_residence)

# Check distribution
print(df_csv['residence_bucket'].value_counts())
print(df_csv['employment_bucket'].value_counts())

residence_bucket
7+ years       508
<6 months      154
6-12 months    154
3-7 years      104
1-3 years       80
Name: count, dtype: int64
employment_bucket
7+ years     354
3-7 years    242
1-3 years    232
<1 year      172
Name: count, dtype: int64
